In [1]:
import pandas as pd
from difflib import SequenceMatcher
import numpy as np

In [40]:
prof_info = pd.read_csv('prof_info.csv',header=None)
prof_info = prof_info[prof_info[0]!=prof_info[3]]
prof_list = pd.read_csv('Pro_list.xlsx')
prof_info.columns = ['Name','Web','Paper','Author','Citation','Year','Discription']

In [41]:
prof_info = prof_info.merge(prof_list[['University','College','Name','GoogleScholar_Name']],how= 'left', left_on='Name',right_on='Name')

In [42]:
def rename_(x,y):
    if not x:
        return y
    return x

In [43]:
prof_info['GoogleScholar_Name'] = prof_info.apply(lambda x: rename_(x['GoogleScholar_Name'],x['Name']), axis=1)

In [44]:
#delete name from university
prof_info.drop('Name',axis = 1,inplace=True)

In [45]:
prof_info.head()

,Web,Paper,Author,Citation,Year,Discription,University,College,GoogleScholar_Name
0,https://scholar.google.com/citations?hl=en&use...,Quantitative brain magnetic resonance imaging ...,"F Xavier Castellanos, Jay N Giedd, Wendy L Mar...",1625.0,1996.0,Background:\nAnatomic magnetic resonance imagi...,NTU,CS,Jagath C Rajapakse
1,https://scholar.google.com/citations?hl=en&use...,Quantitative magnetic resonance imaging of hum...,"Jay N Giedd, John W Snell, Nicholas Lange, Jag...",1226.0,1996.0,Brain magnetic resonance images (MRI) of 104 h...,NTU,CS,Jagath C Rajapakse
2,https://scholar.google.com/citations?hl=en&use...,"Quantitative MRI of the temporal lobe, amygdal...","Jay N Giedd, A Catherine Vaituzis, Susan D Ham...",817.0,1996.0,"The volume of the temporal lobe, superior temp...",NTU,CS,Jagath C Rajapakse
3,https://scholar.google.com/citations?hl=en&use...,Statistical approach to segmentation of single...,"Jagath C Rajapakse, Jay N Giedd, Judith L Rapo...",699.0,1997.0,A statistical model is presented that represen...,NTU,CS,Jagath C Rajapakse
4,https://scholar.google.com/citations?hl=en&use...,Sexual dimorphism of the developing human brain,"Jay N Giedd, F Xavier Castellanos, Jagath C Ra...",549.0,1997.0,1.\n1. Sexual dimorphism of human brain anatom...,NTU,CS,Jagath C Rajapakse


In [46]:
#split name
prof_info['split_name']=prof_info.Author.apply(lambda x : str(x).split(', '))

In [47]:
#similarity function
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()#引用ratio方法，返回序列相似性的度量


In [48]:
#find the most similar name
prof_info['name_in_coauthor']=prof_info.apply(lambda x : np.argmax([ 
similarity(x['GoogleScholar_Name'],i) for i in x['split_name']
])
,axis =1)

In [49]:
def replace_name(orig_name,name_list,ind):
    name_list[ind] = orig_name
    return name_list

In [50]:
prof_info['split_name']=prof_info.apply(lambda x : replace_name(x['GoogleScholar_Name'],x['split_name'],x['name_in_coauthor']),axis=1)

In [51]:
author_name_list = []

In [52]:
for each in prof_info['split_name'].to_list():
    author_name_list.extend(each)

In [53]:
author_name_list = set(author_name_list)

In [54]:
num_to_name=dict(zip(list(range(0,len(author_name_list))),author_name_list))

In [55]:
name_to_num = dict(zip(author_name_list,list(range(0,len(author_name_list)))))

In [56]:
prof_info['author_num'] = prof_info['GoogleScholar_Name'].apply(lambda x : name_to_num[x])
prof_info['author_num_list'] = prof_info['split_name'].apply(lambda x : [name_to_num[i] for i in x])

In [84]:
#edges
all_node1,all_node2 = [],[]

In [85]:
num_list = []
for each in prof_info['author_num_list'].to_list():
    num_list.extend(each)

In [86]:
num_list = pd.Series(num_list)

In [87]:
num_st1=num_list.value_counts().index[num_list.value_counts()<=3]
num_st1 = list(num_st1)

In [88]:
for each_author_list in prof_info['author_num_list']:
    for each_node1 in each_author_list:
        for each_node2 in each_author_list:
            if each_node1 != each_node2 and each_node1 not in num_st1 and each_node2 not in num_st1:
                all_node1.append(each_node1)
                all_node2.append(each_node2)
        

In [89]:
train = pd.DataFrame({
    'id_1':all_node1, 'id_2':all_node2
})

In [90]:
train.drop_duplicates(inplace = True)

In [91]:
train.to_csv('./EdMot/input/cora_edges.csv',index=False,header=True)

In [92]:
prof_info.to_csv('converted.csv',index = False, header= True)

In [93]:
import json
json_str = json.dumps(num_to_name)

In [94]:
data = json.loads(json_str)
with open('num_to_name.json', 'w') as f:
    json.dump(data, f)

In [95]:
data = json.loads(json.dumps(name_to_num))
with open('name_to_num.json', 'w') as f:
    json.dump(data, f)